In [ ]:
# https://www.youtube.com/watch?v=WnBCPG-ZdLk How to setup autogen with docker

In [1]:
from dotenv import load_dotenv
load_dotenv(override=True)

Python-dotenv could not parse statement starting at line 3


True

In [2]:
from autogen import AssistantAgent, UserProxyAgent

# have your openai api key as an environment variable as OPENAI_API_KEY

# create an AssistantAgent instance named "assistant"
assistant = AssistantAgent(name="assistant")

# create a UserProxyAgent instance named "user_proxy"
user_proxy = UserProxyAgent(name="user_proxy")

# the assistant receives a message from the user, which contains the task description
user_proxy.initiate_chat(
    assistant,
    message="""What date is today? Which big tech stock has the largest year-to-date gain this year? How much is the gain?""",
)

user_proxy (to assistant):

What date is today? Which big tech stock has the largest year-to-date gain this year? How much is the gain?

--------------------------------------------------------------------------------
assistant (to user_proxy):

Let's solve this task step by step.

Step 1:
First, let's print the current date. Python can get the current date using the datetime module. Here you go:

```python
import datetime

today = datetime.date.today()
print(f"Today's date is {today}")
```

Step 2:
To find out the big tech stocks with the largest year-to-date gain, we need real-time stock market data. Google, Yahoo, Microsoft, Amazon, and Apple are generally considered the big tech stocks. We need the end of year prices for last year and the current prices to calculate the year-to-date gain.

We can get this information from the Yahoo Finance page for each stock. However, manually browsing each page is time-consuming and error-prone. We need a programmatic way to do this. We can use t

In [1]:
from autogen import AssistantAgent, UserProxyAgent, config_list_from_json

# Load LLM inference endpoints from an env variable or a file
# See https://microsoft.github.io/autogen/docs/FAQ#set-your-api-endpoints
# and OAI_CONFIG_LIST_sample.json
config_list = config_list_from_json(
    env_or_file="OAI_CONFIG_LIST",
    file_location=".",
    filter_dict={
        "model": {
            "gpt-4",
            "gpt-3.5-turbo",
        }
    });

assistant = AssistantAgent("assistant", llm_config={"config_list": config_list})
user_proxy = UserProxyAgent("user_proxy", code_execution_config={"work_dir": "coding"})
user_proxy.initiate_chat(assistant, message="Plot a chart of NVDA and TESLA stock price change YTD.")
# This initiates an automated chat between the two agents to solve the task

user_proxy (to assistant):

Plot a chart of NVDA and TESLA stock price change YTD.

--------------------------------------------------------------------------------
assistant (to user_proxy):

Okay! We'll use the "yfinance" library of Python to fetch the data for these stocks. If you do not have it installed, you can install it using pip. We'll also need "matplotlib" to plot the data.

Here's the Python code that will generate a chart for the Year-to-Date (YTD) stock price change of NVDA and TESLA.

```python
# filename: stock_price_chart.py

import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime

# Define the tickers
tickers = ['NVDA', 'TSLA']

# Get today's date
today = datetime.today().strftime('%Y-%m-%d')

# Get the first date of this year
start_date = datetime.today().strftime('%Y-01-01')

# Fetch the data
data = yf.download(tickers, start=start_date, end=today)

# Plot the Adjusted Close price
data['Adj Close'].plot(figsize=(10,5))

# Set the title an

In [5]:
import autogen

config_list = autogen.config_list_from_json(env_or_file="OAI_CONFIG_LIST.json")

llm_config={
    "request_timeout": 600,
    "seed": 44,                     # for caching and reproducibility
    "config_list": config_list,     # which models to use
    "temperature": 0,               # for sampling
}

agent_assistant = autogen.AssistantAgent(
    name="agent_assistant",
    llm_config=llm_config,
)

agent_proxy = autogen.UserProxyAgent(
    name="agent_proxy",
    human_input_mode="NEVER",           # NEVER, TERMINATE, or ALWAYS 
                                            # TERMINATE - human input needed when assistant sends TERMINATE 
    max_consecutive_auto_reply=10,
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={
        "work_dir": "agent_output",     # path for file output of program
        "use_docker": False,            # True or image name like "python:3" to use docker image
    },
    llm_config=llm_config,
    system_message="""Reply TERMINATE if the task has been solved at full satisfaction.
                      Otherwise, reply CONTINUE, or the reason why the task is not solved yet."""
)

agent_proxy.initiate_chat(agent_assistant, message="Plot a chart of NVDA and TESLA stock price change YTD.")

agent_proxy (to agent_assistant):

Plot a chart of NVDA and TESLA stock price change YTD.

--------------------------------------------------------------------------------
agent_assistant (to agent_proxy):

To plot a chart of NVDA and TESLA stock price change YTD, we can use the `yfinance` library in Python. This library allows us to download stock data from Yahoo Finance, which we can then use to plot the data.

Here is the Python code to do this:

```python
# filename: stock_price_plot.py

import yfinance as yf
import matplotlib.pyplot as plt
import datetime

# Get today's date
today = datetime.date.today()

# Get the date one year ago
one_year_ago = today - datetime.timedelta(days=365)

# Download the stock data
nvda = yf.download('NVDA', start=one_year_ago, end=today)
tsla = yf.download('TSLA', start=one_year_ago, end=today)

# Plot the data
plt.figure(figsize=(14, 7))
plt.plot(nvda['Close'], label='NVDA')
plt.plot(tsla['Close'], label='TSLA')
plt.title('NVDA vs TSLA Stock Price YT